*A demonstration of deploying a thread, partially completing, pausing, and resuming.*

Desired behavior:

<img src="../media/Threads-Concept-01.png" width=500>

Pseudo Code

```
# ----------------------------------------------- Scheduler
for event in future:
    update sim time
    get process based on event
    return control to process
    # ------------------------------------------- Process
    Do process things
    Update process states
    return control to scheduler
    # ------------------------------------------- Scheduler
    update system state
    schedule next event
    start next process?
```

---
### Demo `ascyncio`

In [1]:
import asyncio

In [2]:
async def nested():
    return 42

async def main():
    # Nothing happens if we just call "nested()".
    # A coroutine object is created but not awaited,
    # so it *won't run at all*.
    nested()

    # Let's do it differently now and await it:
    print(await nested())  # will print "42".

# asyncio.run(main())

In [3]:
await nested()

42

Working with `Events`

In [4]:
async def fly_mission(event, name): 
    print(f"{name}: I'm flyin!")

    await event.wait()

    print(f"{name}: I'm done.")

In [5]:
print("Start mission")

# At the same time that we start a mission we will schedule a future event
end_flightA = asyncio.Event()
end_flightB = asyncio.Event()

flightA = asyncio.create_task(fly_mission(end_flightA, "Vehicle A"), name="flightA")
flightB = asyncio.create_task(fly_mission(end_flightB, "Vehicle B"), name="flightB")

fel = [end_flightA, end_flightB]

Start mission
Vehicle A: I'm flyin!
Vehicle B: I'm flyin!


In [6]:
# end_mission.set()

### Introspections -- what are the active tasks?

In [7]:
asyncio.current_task().get_name()

'Task-3'

In [8]:
asyncio.all_tasks()

{<Task pending name='Task-3' coro=<Kernel.dispatch_queue() running at /Users/ben/.pyenv/versions/3.10.3/envs/SpaceDES/lib/python3.10/site-packages/ipykernel/kernelbase.py:473> cb=[IOLoop.add_future.<locals>.<lambda>() at /Users/ben/.pyenv/versions/3.10.3/envs/SpaceDES/lib/python3.10/site-packages/tornado/ioloop.py:688]>,
 <Task pending name='flightA' coro=<fly_mission() running at /var/folders/d4/714vzybx28lcn2xx4n__4n8m0000gp/T/ipykernel_48598/333057138.py:4> wait_for=<Future pending cb=[Task.task_wakeup()]>>,
 <Task pending name='flightB' coro=<fly_mission() running at /var/folders/d4/714vzybx28lcn2xx4n__4n8m0000gp/T/ipykernel_48598/333057138.py:4> wait_for=<Future pending cb=[Task.task_wakeup()]>>}

In [9]:
end_flightB.set()

Vehicle B: I'm done.


In [11]:
end_flightA.set()

Vehicle A: I'm done.


In [10]:
asyncio.all_tasks()

{<Task pending name='Task-3' coro=<Kernel.dispatch_queue() running at /Users/ben/.pyenv/versions/3.10.3/envs/SpaceDES/lib/python3.10/site-packages/ipykernel/kernelbase.py:473> cb=[IOLoop.add_future.<locals>.<lambda>() at /Users/ben/.pyenv/versions/3.10.3/envs/SpaceDES/lib/python3.10/site-packages/tornado/ioloop.py:688]>,
 <Task pending name='flightA' coro=<fly_mission() running at /var/folders/d4/714vzybx28lcn2xx4n__4n8m0000gp/T/ipykernel_48598/333057138.py:4> wait_for=<Future pending cb=[Task.task_wakeup()]>>}

#### Simple Demo

In [ ]:
from dataclasses import dataclass

@dataclass
class Simulation:
    future: list  # of events
    active: list  # of tasks
    complete: list  # of tasks

async def fly_mission(event, name): 
    print(f"{name}: I'm flyin!")

    await event.wait()

    print(f"{name}: I'm done.")

Start an empty sim

In [ ]:
sim = Simulation([], [], [])
sim

Simulation(future=[], active=[], complete=[])

Start a process and schedule an ending event

In [ ]:
# For some vehicle
vehicle = "Vehicle A"

# Get some process pair --> fly_mission, end_mission
# - Create an instance of the ending event
end_mission = asyncio.Event()
# - Create the task object, this schedules the ending event
mission = asyncio.create_task(fly_mission(end_mission, vehicle))

# Update the sim with the events and missions
sim.future.append(end_mission)
sim.active.append(mission)

# check the sim
sim

Simulation(future=[<asyncio.locks.Event object at 0x10493f250 [unset]>], active=[<Task pending name='Task-10' coro=<fly_mission() running at /var/folders/d4/714vzybx28lcn2xx4n__4n8m0000gp/T/ipykernel_71677/3053018486.py:9>>], complete=[])

Vehicle A: I'm flyin!


Now trigger the next event and update the appropriate task

In [ ]:
# Get the next event
next_event = sim.future[0]

next_event.set()

Vehicle A: I'm done.


Process the task status

In [ ]:
completed_task =  sim.active[0]

sim.active.remove(completed_task)
sim.complete.append(completed_task)

sim

Simulation(future=[<asyncio.locks.Event object at 0x10493f250 [set]>], active=[], complete=[<Task finished name='Task-10' coro=<fly_mission() done, defined at /var/folders/d4/714vzybx28lcn2xx4n__4n8m0000gp/T/ipykernel_71677/3053018486.py:9> result=None>])

In [ ]:
@dataclass
class Activity:
    name: str
    start: float
    end: float
    task: asyncio.Task

In [ ]:
end_mission = asyncio.Event()
# - Create the task object, this schedules the ending event
mission = asyncio.create_task(fly_mission(end_mission, vehicle))

a = Activity("Flight", 0, 10, mission)

---

In [ ]:
@dataclass
class Simulation:
    future: list  # of events
    active: list  # of tasks
    complete: list  # of tasks
    
    def schedule(self, coro, event, vehicle):

        new_task = asyncio.create_task(coro(event, vehicle))
        # update the sim state
        self.future.append(event)
        self.active.append(new_task)

        new_task.add_done_callback(self.update_state)

        # self.complete(new_task)

    def update_state(self, task):

        self.future.remove(event)
        self.active.remove(task)
        self.complete.append(task)


In [ ]:
async def fly_mission(event, name): 
    print(f"{name}: I'm flyin!")

    await event.wait()

    print(f"{name}: I'm done.")

In [ ]:
sim = Simulation([], [], [])

vehicle = "Vehicle A"

event = asyncio.Event()

sim.schedule(fly_mission, event, vehicle)

sim

Simulation(future=[<asyncio.locks.Event object at 0x103d5aa40 [unset]>], active=[<Task pending name='Task-26' coro=<fly_mission() running at /var/folders/d4/714vzybx28lcn2xx4n__4n8m0000gp/T/ipykernel_71677/333057138.py:1> cb=[Simulation.update_state()]>], complete=[])

Vehicle A: I'm flyin!


In [ ]:
event.set()

Vehicle A: I'm done.


In [ ]:
sim

Simulation(future=[], active=[], complete=[<Task finished name='Task-26' coro=<fly_mission() done, defined at /var/folders/d4/714vzybx28lcn2xx4n__4n8m0000gp/T/ipykernel_71677/333057138.py:1> result=None>])